In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> index

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

C:\Users\Sunghee\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0 loss: 2.3129354 Prediction: ooooouuououooou
1 loss: 2.1707573 Prediction: oo  ou   ouooou
2 loss: 2.010143 Prediction: yo  ou      you
3 loss: 1.7764989 Prediction: yo  ou   yyyyou
4 loss: 1.4996293 Prediction: yo oou   ytyyou
5 loss: 1.2301306 Prediction: yo oou   nt you
6 loss: 0.99280465 Prediction: yo you wnnt you
7 loss: 0.7827123 Prediction: yf you want you
8 loss: 0.5843908 Prediction: yf you want you
9 loss: 0.44793317 Prediction: yf you want you
10 loss: 0.32965234 Prediction: yf you want you
11 loss: 0.24888042 Prediction: yf you want you
12 loss: 0.18980058 Prediction: yf you want you
13 loss: 0.1409609 Prediction: if you want you
14 loss: 0.100945964 Prediction: if you want you
15 loss: 0.070841655 Prediction: if you want you
16 loss: 0.049419604 Prediction: if you want you
17 loss: 0.03462091 Prediction: if you want you
18 loss: 0.024686625 Prediction: if you want you
19 loss: 0.018126056 Prediction: if you want you
20 loss: 0.013756123 Prediction: if you want you
21 los